In [1]:
import sys
sys.path.insert(0, 'env')
sys.path.insert(0, 'lib')

import ccg
from agents import ARagent
import pandas as pd
import numpy as np
import copy
import random
import torch
seed = 12345644

np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

cards = pd.read_csv('env/configue/cardsTable.csv')
cardsList = [ccg.Minion(i) for i in cards.values.tolist()]

cores = pd.read_csv('env/configue/cores.csv')
coreList = [ccg.Core(i) for i in cores.values.tolist()]

playersNum = 2
piles = [[ccg.Pile(cardsList, 10) for _ in range(4)] for I in range(playersNum)]
cores = np.random.choice(coreList, 2)
decks = [ccg.Deck(cores[i], piles[i], i) for i in range(playersNum)]

In [2]:
session = ccg.Session(decks)

In [3]:
from __future__ import print_function
print(torch.__version__)

0.4.1


In [4]:
from trainer import Trainer

trainer = Trainer(session, [ARagent(), ARagent()])

In [7]:
from IPython.display import clear_output

wins = dict()
wins[-1] = 0
wins[0] = 0
wins[1] = 0
for i in range(1000):
    session.reset()
    trainer.playGame()
    observation, _, _ = session.processNewStateInfo()
    winner = -1
    if(observation["loser"] != -1):
        winner = 1 - observation["loser"]
    wins[winner] += 1
    if(i % 10 == 0):
        clear_output()
        print(i)

990


In [8]:
wins

{-1: 930, 0: 32, 1: 38}